In [0]:
pip install findspark

Python interpreter will be restarted.
Collecting findspark
 Using cached findspark-1.4.2-py2.py3-none-any.whl (4.2 kB)
Installing collected packages: findspark
Successfully installed findspark-1.4.2
Python interpreter will be restarted.

In [0]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import StandardScaler, VectorAssembler,VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
dataset = spark.read.csv("/FileStore/tables/Admission_Prediction.csv",header=True)

In [0]:
dataset.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
 337.00| 118.00| 4|4.50|4.50|9.65| 1.00| 0.92|
 324.00| 107.00| 4|4.00|4.50|8.87| 1.00| 0.76|
 null| 104.00| 3|3.00|3.50|8.00| 1.00| 0.72|
 322.00| 110.00| 3|3.50|2.50|8.67| 1.00| 0.80|
 314.00| 103.00| 2|2.00|3.00|8.21| 0.00| 0.65|
 330.00| 115.00| 5|4.50|3.00|9.34| 1.00| 0.90|
 321.00| 109.00| null|3.00|4.00|8.20| 1.00| 0.75|
 308.00| 101.00| 2|3.00|4.00|7.90| 0.00| 0.68|
 302.00| 102.00| 1|2.00|1.50|8.00| 0.00| 0.50|
 323.00| 108.00| 3|3.50|3.00|8.60| 0.00| 0.45|
 325.00| 106.00| 3|3.50|4.00|8.40| 1.00| 0.52|
 327.00| 111.00| 4|4.00|4.50|9.00| 1.00| 0.84|
 null| 112.00| 4|4.00|4.50|9.10| 1.00| 0.78|
 307.00| 109.00| 3|4.00|3.00|8.00| 1.00| 0.62|
 311.00| 104.00| 3|3.50|2.00|8.20| 1.00| 0.61|
 314.00| 105.00| 3|3.50|2.50|8.30| 0.00| 0.54|
 317.00| 107.00| 3|4.00|3.00|8.70| 0.00| 0.66|
 319.00| 106.00| 3|4.00|3.00|8.00| 1.00| 0.65|
 318.00| 110.00| 3|4.00|3.00|8.80| 0.00| 0.63|
 303.00| 102.00| 3|3.50|3.00|8.50| 0.00| 0.62|
+---------+-----------+-----------------+----+----+----+--------+---------------+
only showing top 20 rows

In [0]:
dataset.printSchema()

root
-- GRE Score: string (nullable = true)
-- TOEFL Score: string (nullable = true)
-- University Rating: string (nullable = true)
-- SOP: string (nullable = true)
-- LOR: string (nullable = true)
-- CGPA: string (nullable = true)
-- Research: string (nullable = true)
-- Chance of Admit: string (nullable = true)

In [0]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [0]:
new_data.printSchema()

root
-- GRE Score: float (nullable = true)
-- TOEFL Score: float (nullable = true)
-- University Rating: float (nullable = true)
-- SOP: float (nullable = true)
-- LOR: float (nullable = true)
-- CGPA: float (nullable = true)
-- Research: float (nullable = true)
-- Chance of Admit: float (nullable = true)

In [0]:
from pyspark.sql.functions import col, count, isnan, when

In [0]:
#checking for null if nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 15| 10| 15| 0| 0| 0| 0| 0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

In [0]:
from pyspark.ml.feature import Imputer

In [0]:
imputer = Imputer(inputCols=["GRE Score", "TOEFL Score","University Rating"], 
                  outputCols=["GRE Score", "TOEFL Score","University Rating"])
model = imputer.fit(new_data)

imputed_data = model.transform(new_data)

In [0]:
#checking for null if nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

In [0]:
features = imputed_data.drop('Chance of Admit')

In [0]:
#let's assemble our features together using vectorAssembler
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [0]:
output = assembler.transform(imputed_data)

In [0]:
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(output)

In [0]:
featureIndexer = featureIndexer.transform(output)

In [0]:
new_indexed_data = featureIndexer.select("indexedFeatures", "Chance of Admit")

In [0]:
training, test = new_indexed_data.randomSplit([0.7, 0.3])

In [0]:
training.show()

+--------------------+---------------+
 indexedFeatures|Chance of Admit|
+--------------------+---------------+
[290.0,100.0,1.0,...| 0.47|
[290.0,104.0,4.0,...| 0.45|
[293.0,97.0,2.0,2...| 0.64|
[294.0,93.0,1.0,1...| 0.46|
[294.0,95.0,1.0,1...| 0.49|
[295.0,96.0,2.0,1...| 0.47|
[295.0,99.0,2.0,2...| 0.57|
[295.0,101.0,2.0,...| 0.69|
[296.0,97.0,2.0,1...| 0.49|
[296.0,99.0,2.0,2...| 0.61|
[296.0,101.0,1.0,...| 0.6|
[297.0,96.0,2.0,2...| 0.43|
[297.0,96.0,2.0,2...| 0.34|
[297.0,98.0,2.0,2...| 0.59|
[298.0,97.0,3.121...| 0.45|
[298.0,98.0,2.0,1...| 0.44|
[298.0,98.0,2.0,4...| 0.34|
[298.0,99.0,1.0,1...| 0.53|
[298.0,100.0,3.0,...| 0.58|
[298.0,101.0,4.0,...| 0.53|
+--------------------+---------------+
only showing top 20 rows

In [0]:
gbt = GBTRegressor(featuresCol="indexedFeatures",labelCol="Chance of Admit", maxIter=10)

In [0]:
# Train model.  This also runs the indexer.
model = gbt.fit(training)

In [0]:
# Make predictions.
predictions = model.transform(test)

In [0]:
predictions.show()

+--------------------+---------------+-------------------+
 indexedFeatures|Chance of Admit| prediction|
+--------------------+---------------+-------------------+
[295.0,93.0,1.0,2...| 0.46|0.44674724535254856|
[295.0,99.0,1.0,2...| 0.37| 0.4819637393704025|
[296.0,95.0,2.0,3...| 0.44|0.49471374817864544|
[296.0,99.0,2.0,3...| 0.47| 0.5500160298645194|
[297.0,99.0,4.0,3...| 0.54|0.49385304534628466|
[297.0,100.0,1.0,...| 0.52| 0.5837531827675491|
[297.0,101.0,3.0,...| 0.57| 0.48639433606417|
[298.0,92.0,1.0,2...| 0.51| 0.4707630378818926|
[298.0,101.0,2.0,...| 0.54| 0.5566346655725719|
[298.0,105.0,3.0,...| 0.69| 0.6693261972812755|
[299.0,94.0,1.0,1...| 0.42|0.41928497349717175|
[299.0,100.0,3.0,...| 0.63| 0.5631441390151829|
[299.0,102.0,3.0,...| 0.56| 0.6296446807383567|
[300.0,99.0,1.0,1...| 0.58| 0.594743737272896|
[300.0,99.0,1.0,3...| 0.36| 0.5194542946294879|
[300.0,100.0,3.0,...| 0.64| 0.7897431595789103|
[300.0,102.0,3.0,...| 0.63| 0.6250953524894749|
[301.0,96.0,1.0,3...| 0.54| 0.6562479316711531|
[301.0,98.0,1.0,2...| 0.67| 0.6222495075892991|
[301.0,102.0,3.0,...| 0.68| 0.5882068659658466|
+--------------------+---------------+-------------------+
only showing top 20 rows

In [0]:
evaluator = RegressionEvaluator(
    labelCol="Chance of Admit", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print ("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.0712873

In [0]:
spark.stop()